# Little Lemon analysis and sales report

### Task 1
Complete the following steps to establish a connection pool:

- To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

- To find the information on the error, import the Error class from MySQL Connector/Python.

- Define your database configurations as a Python dictionary object called dbconfig.

- Establish a connection pool [pool_name = pool_b] with two connections. 

- Implement error handling using a try-except block in case the connection fails. 

In [39]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database": "little_lemon_db",
    "user": "meta",
    "password": "password"
}

try:
    pool = MySQLConnectionPool(
        pool_name = "pool_b",
        pool_size = 2,
        **dbconfig
    )
    print("Pool created with name: {} and size: {}".format(pool.pool_name, pool.pool_size))
    
except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

Pool created with name: pool_b and size: 2


### Task 2
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:


TIP: You need to add a connection to connect the third guest.


Guest 1:

- Table Number: 8

- First Name: Anees

- Last Name: Java

- Booking Time: 18:00

- EmployeeID: 6


Guest 2:

- Table Number: 5

- First Name: Bald

- Last Name: Vin

- Booking Time: 19:00

- EmployeeID: 6


Guest 3:

- Table Number: 12

- First Name: Jay

- Last Name: Kon

- Booking Time: 19:30 

- EmployeeID: 6


Return all the connections back to the pool. 


TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message.  

In [40]:
from mysql.connector import connect
from mysql.connector.pooling import PoolError

data = [
    (8, 'Anees', 'Java', '18:00', 6),
    (8, 'Bald', 'Vin', '19:00', 6),
    (8, 'Jay', 'Kon', '19:30', 6),
]

try:
    connections, cursors = [], []

    for i, booking in enumerate(data):

        insert_query = f"""
            INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
            VALUES (%s, %s, %s, %s, %s);
        """

        try:
            connection = pool.get_connection()
            connections.append(connection)
            print(f"Connection obtained for Guest {i+1}")

            cursor = connection.cursor()
            cursors.append(cursor)
            print(f"Cursor obtained for Guest {i+1}")

            cursor.execute(insert_query, booking)
            connection.commit()
            print(f"Booking inserted for Guest {i+1} with connection from pool")

        except PoolError as pe:
            connection = connect(
                **dbconfig
            )
            connections.append(connection)
            cursors.append(cursor)

            cursor.execute(insert_query, booking)
            connection.commit()
            print(f"Booking inserted for Guest {i+1} with new connection")

finally:
    for i, (cursor, connection) in enumerate(zip(cursors, connections)):
        if connection.is_connected():
            cursor.close()
            try:
                connection.close()
                print(f"Connection {i+1} returned to pool")
            except PoolError as pe:
                print(f"Cannot return connection {i} to pool: {pe}")


Connection obtained for Guest 1
Cursor obtained for Guest 1
Booking inserted for Guest 1 with connection from pool
Connection obtained for Guest 2
Cursor obtained for Guest 2
Booking inserted for Guest 2 with connection from pool
Booking inserted for Guest 3 with new connection
Connection 1 returned to pool
Connection 2 returned to pool
Connection 3 returned to pool


### Task 3
Create a report containing the following information:

- The name and EmployeeID of the Little Lemon manager.

- The name and role of the employee who receives the highest salary.

- The number of guests booked between 18:00 and 20:00.

- The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [48]:
queries = {
    'get_manager': """
        SELECT Name, EmployeeID
        FROM Employees
        WHERE Role = 'Manager';
    """,
    'highest_salary_employee': """
        SELECT Name, Role
        FROM Employees
        WHERE Annual_Salary = (
            SELECT MAX(Annual_Salary)
            FROM Employees
        );
    """,
    'count_guests': """
        SELECT COUNT(BookingID) as GuestCount
        FROM Bookings
        WHERE TIME(BookingSlot) BETWEEN '18:00' AND '20:00';
    """,
    'waiting_guests': """
        SELECT
            CONCAT(B.GuestFirstName, ' ', B.GuestLastName) as FullName,
            B.BookingID AS BookingID
        FROM Bookings AS B
        INNER JOIN Employees AS E ON B.EmployeeID = E.EmployeeID
        WHERE E.Role = 'Receptionist'
        ORDER BY B.BookingSlot;
    """
}

results = {}

# Get results

try:
    connection = pool.get_connection()
    cursor = connection.cursor(dictionary=True)

    for query_name, query in queries.items():
        cursor.execute(query)
        results[query_name] = cursor.fetchall()

except Exception as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)
finally:
    if connection.is_connected():
        cursor.close()
        print("The cursor is closed")
        connection.close()
        print("The connection is closed")
    else:
        print("The connection is already closed")

# Generate report
print("Little Lemon Restaurant Report")
print("------------------------------")

if 'get_manager' in results:
    result = results['get_manager'][0]
    print(f"Little Lemon Manager name: {result['Name']}, employee ID: {result['EmployeeID']}")

if 'highest_salary_employee' in results:
    result = results['highest_salary_employee'][0]
    print(f"Employee who receives the highest salary name: {result['Name']}, role: {result['Role']}")

if 'count_guests' in results:
    result = results['count_guests'][0]
    print(f"The number of guests booked between 18:00 and 20:00: {result['GuestCount']}")

if 'waiting_guests' in results:
    result = results['waiting_guests'][0]
    print("Waiting guests:")
    for guest in results['waiting_guests']:
        print(f"- name: {guest['FullName']}, booking ID: {guest['BookingID']}")

The cursor is closed
The connection is closed
Little Lemon Restaurant Report
------------------------------
Little Lemon Manager name: Mario Gollini, employee ID: 1
Employee who receives the highest salary name: Mario Gollini, role: Manager
The number of guests booked between 18:00 and 20:00: 6
Waiting guests:
- name: Anees Java, booking ID: 7
- name: Anees Java, booking ID: 8
- name: Anees Java, booking ID: 11
- name: Bald Vin, booking ID: 12


### Task 4
Create a stored procedure named BasicSalesReport that returns the following statistics: 

- Total sales

- Average sale

- Minimum bill paid

- Maximum bill paid

In [55]:
proc_sales_report = """
CREATE PROCEDURE BasicSalesReport()
BEGIN

    SELECT
        SUM(BillAmount) AS TotalSales,
        AVG(BillAmount) AS AverageSale,
        MIN(BillAmount) AS MinimumBill,
        MAX(BillAmount) AS MaximumBill
    FROM Orders;

END
"""

try:
    connection = pool.get_connection()
    cursor = connection.cursor(dictionary=True)

    cursor.execute(proc_sales_report)

    cursor.callproc("BasicSalesReport")

    results = next(cursor.stored_results())
    data = results.fetchall()[0]

    for col, val in data.items():
        print(col + ' ' + str(val))

except Exception as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

finally:
    if connection.is_connected():
        cursor.close()
        print("The cursor is closed")
        connection.close()
        print("The connection is closed")
    else:
        print("The connection is already closed")

TotalSales 243
AverageSale 48.6000
MinimumBill 37
MaximumBill 86
The cursor is closed
The connection is closed


### Task 5
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

- Get a connection from the pool.

- Create a buffered cursor.

- Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

- Returned the connection back to the pool.


The output should be as follows:

`[BookingSlot]`

`[Guest_name]`

`[Assigned to: Employee Name [Employee Role]]`

In [58]:
upcoming_query = """
SELECT
    b.BookingSlot AS BookingSlot,
    CONCAT(b.GuestFirstName, ' ', b.GuestLastName) AS GuestName,
    CONCAT('Assigned to: ', e.Name, ' [', e.Role, ']') AS EmployeeData
FROM Bookings AS b
INNER JOIN Employees AS e ON b.EmployeeID = e.EmployeeID
ORDER BY BookingSlot ASC
LIMIT 3
"""

try:
    connection = pool.get_connection()
    cursor = connection.cursor(buffered=True)

    cursor.execute(upcoming_query)

    for result in cursor.fetchall():
        print(f"{result[0]}")
        print(f"{result[1]}")
        print(f"{result[2]}")


finally:
    if connection.is_connected():
        cursor.close()
        print("The cursor is closed")
        connection.close()
        print("The connection is closed")
    else:
        print("The connection is already closed")

15:00:00
Vanessa McCarthy
Assigned to: Giorgos Dioudis [Head Chef]
17:00:00
Anees Java
Assigned to: John Millar [Receptionist]
17:30:00
Marcos Romero
Assigned to: Fatma Kaya [Assistant Chef]
The cursor is closed
The connection is closed
